In [1]:
import pandas as pd
import numpy as np
from kafka import KafkaProducer
from kafka import KafkaConsumer
import json
import time
import logging
import pickle
from pyspark.sql.functions import *
import csv

In [2]:
topic_name = "clickstream_data"
kafka_broker = "localhost:9092"

In [3]:
def create_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers=kafka_broker, 
            value_serializer=lambda x:json.dumps(x).encode('utf-8'),
            key_serializer=lambda x:json.dumps(x).encode('utf-8'),
        )
    except Exception as e:
        logging.exception("Couldn't create the producer")
        producer = None
    
    return producer

def create_consumer(topic_name):
    try:
        consumer = KafkaConsumer(
        topic_name,
        bootstrap_servers=kafka_broker,
        auto_offset_reset="earliest",
        enable_auto_commit=True,
        value_deserializer=lambda x: json.loads(x.decode("utf-8")))
    except Exception as e:
        logging.exception("Couldn't create the consumer")
        consumer = None
    return consumer

In [4]:
with open("../streamlit/unique_std.pkl", 'rb') as f:
  unique_std = pickle.load(f)
len(unique_std)

411

In [5]:
send_data = pd.read_csv("../public data/data_a_hidden_10.csv")
send_data['EventTime'] = pd.to_datetime(send_data['EventTime'])
selected_student_ids = send_data.STUDENTID.unique()[:3]
#selected_student_ids = np.random.choice(send_data['STUDENTID'].unique(), 3, replace=False)
random_std = pd.DataFrame()
for student_id in selected_student_ids:
    student_data = send_data[send_data.STUDENTID == student_id].head(10)
    random_std = pd.concat([random_std, student_data])

random_std['value_sort'] = random_std['EventTime'].dt.strftime("%H:%M:%S.%f")
random_std = random_std.sample(frac=1).reset_index(drop=True)
random_std = random_std.sort_values(by='value_sort', ascending=True).reset_index(drop=True)
random_std.drop(columns='value_sort', inplace=True)
random_std['EventTime'] = random_std['EventTime'].astype(str) ## lưu ý check kỹ trước khi send to producer

In [7]:
producer = create_producer()
for _,event in random_std.iterrows():
    data = event.to_json()
    print(data)
    producer.send(topic_name, value=data)
    time.sleep(2)
    producer.flush()
producer.close()

{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH356862","ItemType":"Directions","Observable":"Enter Item","ExtendedInfo":null,"EventTime":"2017-02-08 14:13:25.390"}
{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH356862","ItemType":"Directions","Observable":"Next","ExtendedInfo":null,"EventTime":"2017-02-08 14:13:34.937"}
{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH356862","ItemType":"Directions","Observable":"Exit Item","ExtendedInfo":null,"EventTime":"2017-02-08 14:13:34.947"}
{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH098810","ItemType":"MCSS","Observable":"Enter Item","ExtendedInfo":null,"EventTime":"2017-02-08 14:13:35.087"}
{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH098810","ItemType":"MCSS","Observable":"Click Choice","ExtendedInfo":"VH098810_4:checked","EventTime":"2017-02-08 14:13:44.400"}
{"STUDENTID":2333001731,"Block":"A","AccessionNumber":"VH098519","ItemType":"MCSS","Observable":"Click Progress Navigator","Extende

In [ ]:
# producer = create_producer()
# with open("./public data/data_a_hidden_10.csv", 'r') as f:
#     csv_reader = csv.reader(f)
#     header = next(csv_reader, None)
#     for row in csv_reader:
#         message = ','.join(row)
#         print(message)
#         producer.send(topic_name, value=message)
#         time.sleep(2)
#         producer.flush()